# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 52 new papers today
          19 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/18 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.08440


extracting tarball to tmp_2211.08440...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.08487


extracting tarball to tmp_2211.08487...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.09071


extracting tarball to tmp_2211.09071...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.09080


extracting tarball to tmp_2211.09080...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.08487-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.08487) | **Calibration of hybrid resolved star formation rate recipes based on  PHANGS-MUSE H$α$ and H$β$ maps**  |
|| Francesco Belfiore, et al. -- incl., <mark>Kathryn Kreckel</mark>, <mark>Lukas Neumann</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2022-11-17*|
|*Comments*| *accepted in A&A*|
|**Abstract**| Mapping star-formation rates (SFR) within galaxies is key to unveiling their assembly and evolution. Calibrations exist for computing SFR from a combination of ultraviolet and infrared bands for galaxies as integrated systems, but their applicability to sub-galactic (kpc) scales remains largely untested. Here we use integral field spectroscopy of 19 nearby ($D <$ 20 Mpc) galaxies obtained by PHANGS-MUSE to derive accurate Balmer decrements (H$\alpha$/H$\beta$) and attenuation-corrected H$\alpha$ maps. We combine this information with mid-infrared maps from WISE at 22 $\rm \mu m$, and ultraviolet maps from GALEX in the far-UV band, to derive SFR surface densities in nearby galaxies on resolved (kpc) scales. Using the H$\alpha$ attenuation-corrected SFR as a reference, we find that hybrid recipes from the literature overestimate the SFR in regions of low SFR surface density, low specific star-formation rate (sSFR), low attenuation and old stellar ages. We attribute these trends to heating of the dust by old stellar populations (IR cirrus). We calibrate this effect by proposing functional forms for the coefficients in front of the IR term which depend on band ratios sensitive to the sSFR. These calibrations prove reliable as a function of physical scale. In particular, they agree within 10% with the attenuation corrections computed from the Balmer decrement on 100 pc scales. Despite small quantitative differences, our calibrations are also applicable to integrated galaxy scales probed by the MaNGA survey, albeit with a larger scatter (up to 0.22 dex). Observations with JWST open up the possibility to calibrate these relations in nearby galaxies with cloud-scale ($\sim$100 pc) resolution mid-IR imaging. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.08493-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.08493) | **The MUSE Hubble Ultra Deep Field surveys: Data release II**  |
|| Roland Bacon, et al. -- incl., <mark>Leindert Boogaard</mark>, <mark>Yucheng Guo</mark>, <mark>Christian Herenz</mark> |
|*Appeared on*| *2022-11-17*|
|*Comments*| *46 pages, 48 figures*|
|**Abstract**| We present the second data release of the MUSE Hubble UDF surveys, which includes the deepest spectroscopic survey ever performed. The MUSE data, with their 3D content, amazing depth, wide spectral range, and excellent spatial and medium spectral resolution, are rich in information. This update of the first release incorporates a new 141-hour adaptive-optics-assisted MXDF field (1' diameter FoV) in addition to the reprocessed 10-hour mosaic (3'x3') and the single 31-hour deep field (1'x1'). We have securely identified and measured the redshift of 2221 sources, an increase of 41% compared to the first release. With the exception of 8 stars, the collected sample consists of 25 nearby galaxies (z < 0.25), 677 OII emitters (z=0.25-1.5), 201 galaxies in the MUSE redshift desert range (z=1.5-2.8), and 1308 LAEs (z=2.8-6.7). This represents an order of magnitude more redshifts than the collection of all spectroscopic redshifts obtained before MUSE in the Hubble UDF area (2221 vs 292). At z > 3, the difference is even more striking, with a factor of 65 increase (1308 vs 20). We compared the measured redshifts against three published photometric redshift catalogs and find the photo-z accuracy to be lower than the constraints provided by photo-z fitting codes. 80% of the galaxies have an HST counterpart. They are on average faint, with a median magnitude of 25.7 and 28.7 for the OII and Ly-alpha emitters, respectively. SED fits show that these galaxies tend to be low-mass star-forming galaxies, with a median stellar mass of 6.2 10**8 M and a median SFR of 0.4 M/yr. 20% of our catalog, or 424 galaxies, have no HST counterpart. The vast majority of these new sources are high EQW z>2.8 LAEs that are detected by MUSE thanks to their bright and asymmetric broad Ly-alpha line. We release advanced data products, specific software, and a web interface to select and download data sets. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.08440-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.08440) | **The White Dwarf Binary Pathways Survey -- IX. Three long period white  dwarf plus subgiant binaries**  |
|| S. G. Parsons, et al. -- incl., <mark>M. S. Hernandez</mark>, <mark>M. R. Schreiber</mark> |
|*Appeared on*| *2022-11-17*|
|*Comments*| *13 pages, 9 figures. Accepted for publication in MNRAS*|
|**Abstract**| Virtually all binaries consisting of a white dwarf with a non-degenerate companion can be classified as either close post-interaction systems (with orbital periods of a few days or less), or wide systems (with periods longer than decades), in which both components have effectively evolved as single stars. Binaries with periods between these two extremes can help constrain common envelope efficiency, or highlight alternative pathways towards the creation of compact binaries. To date such binaries have remained mostly elusive. Here we present three white dwarfs in binaries with evolved subgiant stars with orbital periods of 41, 52 and 461 d. Using Hubble Space Telescope spectroscopy we find that all three systems contain low mass white dwarfs ($\leq$0.4 M$_{\odot}$). One system, TYC 8394$-$1331$-$1, is the inner binary of a hierarchical triple, where the white dwarf plus subgiant binary is orbited by a more distant companion star. These binaries were likely formed from a phase of stable but non-conservative mass transfer, as opposed to common envelope evolution. All three systems will undergo a common envelope phase in the future, but the two shorter period systems are expected to merge during this event, while the longest period system is likely to survive and create a close binary with two low mass white dwarfs. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.08990-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.08990) | **Recovering Galaxy Cluster Convergence from Lensed CMB with Generative  Adversarial Networks**  |
|| <mark>Liam Parker</mark>, Dongwon Han, Pablo Lemos Portela, Shirley Ho |
|*Appeared on*| *2022-11-17*|
|*Comments*| **|
|**Abstract**| We present a new method which leverages conditional Generative Adversarial Networks (cGAN) to reconstruct galaxy cluster convergence from lensed CMB temperature maps. Our model is constructed to emphasize structure and high-frequency correctness relative to the Residual U-Net approach presented by Caldeira, et. al. (2019). Ultimately, we demonstrate that while both models perform similarly in the no-noise regime (as well as after random off-centering of the cluster center), cGAN outperforms ResUNet when processing CMB maps noised with 5uK/arcmin white noise or astrophysical foregrounds (tSZ and kSZ); this out-performance is especially pronounced at high l, which is exactly the regime in which the ResUNet under-performs traditional methods. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.09030-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.09030) | **Angular Diameters and Fundamental Parameters of Forty-Four Stars from  the Navy Precision Optical Interferometer**  |
|| Ellyn K. Baines, et al. -- incl., <mark>Henrique R. Schmitt</mark> |
|*Appeared on*| *2022-11-17*|
|*Comments*| *13 pages, 3 figures, 6 tables. arXiv admin note: substantial text overlap with arXiv:1712.08109*|
|**Abstract**| We measured the angular diameters of 44 stars with the Navy Precision Optical Interferometer, obtaining uncertainties on the limb darkened diameter of 2% or less for all but four stars. We then used our diameters with Gaia or Hipparcos parallaxes to calculate each star's physical radius. We gathered information from the literature to determine bolometric flux and luminosity, and combined that with our diameters to produce an effective temperature. Our sample consists of mostly giant stars, and spans a wide range of spectral classes from B to M. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.09049-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.09049) | **Deep Synoptic Array science I: discovery of the host galaxy of FRB  20220912A**  |
|| Vikram Ravi, et al. -- incl., <mark>Liam Connor</mark>, <mark>Greg Hellbourg</mark>, <mark>Kritti Sharma</mark> |
|*Appeared on*| *2022-11-17*|
|*Comments*| *10 pages, 7 figures, 2 tables, submitted to AAS Journals*|
|**Abstract**| We report the detection and interferometric localization of the repeating fast radio burst (FRB) source FRB 20220912A during commissioning observations with the Deep Synoptic Array (DSA-110). Two bursts were detected from FRB 20220912A, one each on 2022 October 18 and 2022 October 25. The best-fit position is (R.A. J2000, decl. J2000) = (23:09:04.9, +48:42:25.4), with a 90% confidence error ellipse of $\pm2$ arcsec and $\pm1$ arcsec in right ascension and declination respectively. The two bursts have disparate polarization properties and temporal profiles. We find a Faraday rotation measure that is consistent with the low value of $+0.6$ rad m$^{-2}$ reported by CHIME/FRB. The DSA-110 localization overlaps with the galaxy PSO J347.2702+48.7066 at a redshift $z=0.0771$, which we identify as the likely host. PSO J347.2702$+$48.7066 has a stellar mass of approximately $10^{10}M_{\odot}$, modest internal dust extinction, and a star-formation rate likely in excess of $0.1\,M_{\odot}$ yr$^{-1}$. The host-galaxy contribution to the dispersion measure is likely $\lesssim50$ pc cm$^{-3}$. The FRB 20220912A source is therefore likely viewed along a tenuous plasma column through the host galaxy. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.09071-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.09071) | **Observability of signatures of transport-induced chemistry in clear  atmospheres of hot gas giant exoplanets**  |
|| Maria Zamyatina, et al. -- incl., <mark>Duncan A. Christie</mark> |
|*Appeared on*| *2022-11-17*|
|*Comments*| *accepted to MNRAS, 27 pages, 8+7 figures*|
|**Abstract**| Transport-induced quenching, i.e., the homogenisation of chemical abundances by atmospheric advection, is thought to occur in the atmospheres of hot gas giant exoplanets. While some numerical modelling of this process exists, the three-dimensional nature of transport-induced chemistry is underexplored. Here we present results of 3D cloud- and haze-free simulations of the atmospheres of HAT-P-11b, HD 189733b, HD 209458b, and WASP-17b including coupled hydrodynamics, radiative transfer and chemistry. Our simulations were performed with two chemical schemes: a chemical kinetics scheme, which is capable of capturing transport-induced quenching, and a simpler, more widely used chemical equilibrium scheme. We find that transport-induced quenching is predicted to occur in atmospheres of all planets in our sample; however, the extent to which it affects their synthetic spectra and phase curves varies from planet to planet. This implies that there is a "sweet spot" for the observability of signatures of transport-induced quenching, which is controlled by the interplay between the dynamics and chemistry. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.09080-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.09080) | **Breaking Degeneracies in Formation Histories by Measuring Refractory  Content in Gas Giants**  |
|| Yayaati Chachan, <mark>Heather A. Knutson</mark>, Joshua Lothringer, Geoffrey A. Blake |
|*Appeared on*| *2022-11-17*|
|*Comments*| *resubmitted to ApJ after addressing referee's comments*|
|**Abstract**| Relating planet formation to atmospheric composition has been a long-standing goal of the planetary science community. So far, most modeling studies have focused on predicting the enrichment of heavy elements and the C/O ratio in giant planet atmospheres. Although this framework provides useful constraints on the potential formation locations of gas giant exoplanets, carbon and oxygen measurements alone are not enough to determine where a given gas giant planet originated. Here, we show that characterizing the abundances of refractory elements (e.g., silicon, iron) can break these degeneracies. Refractory elements are present in the solid phase throughout most of the disk and their atmospheric abundances therefore reflect the solid-to-gas accretion ratio during formation. We introduce a new framework that parameterizes the atmospheric abundances of gas giant exoplanets in the form of three ratios: Si/H, O/Si, and C/Si. Si/H traces the solid-to-gas accretion ratio of a planet and is loosely equivalent to earlier notions of 'metallicity'. For O/Si and C/Si, we present a global picture of their variation with distance and time based on what we know from the solar system meteorites and an updated understanding of the variations of thermal processing within protoplanetary disks. We show that ultra-hot Jupiters are ideal targets for atmospheric characterization studies using this framework, as we can measure the abundances of refractories, oxygen and carbon in the gas phase. Finally, we propose that hot Jupiters with silicate clouds and low water abundances might have accreted their envelopes between the soot line and the water snowline. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2211.08487.md
    + _build/html/tmp_2211.08487/./Fig/Fig5.png
    + _build/html/tmp_2211.08487/./Fig/Fig7.png
    + _build/html/tmp_2211.08487/./Fig/Fig4.png
exported in  _build/html/2211.08493.md
    + _build/html/tmp_2211.08493/paper_figures/src7636-6294.png
    + _build/html/tmp_2211.08493/paper_figures/skyspec.png
    + _build/html/tmp_2211.08493/paper_figures/process.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\comment}[2][alizarin]{{\color{#1}[[{\bf #2}]]}}$
$\newcommand{\ppxf}{pPXF}$
$\newcommand{\oiii}{\textup{[O \textsc{iii}]}}$
$\newcommand{\nii}{\textup{[N \textsc{ii}]}}$
$\newcommand{\sii}{\textup{[S \textsc{ii}]}}$
$\newcommand{\oi}{\textup{[O \textsc{i}]}}$
$\newcommand{\siii}{\textup{[S \textsc{iii}]}}$
$\newcommand{\oii}{\textup{[O \textsc{ii}]}}$
$\newcommand{\hii}{\textup{H} \textsc{ii}}$
$\newcommand{\ha}{\textup{H}\ensuremath{\alpha}}$
$\newcommand{\hb}{\textup{H}\ensuremath{\beta}}$
$\newcommand{\re}{\textup{R}\ensuremath{_\mathrm{e}}}$
$\newcommand{\Te}{\textup{T}\ensuremath{_\mathrm{e}}}$
$\newcommand{\SFR}{\ensuremath{\Sigma_{\mathrm{SFR}}}}$
$\newcommand{\aha}{\ensuremath{A_{\mathrm{ H \alpha}}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\comment$}[2][alizarin]{{\color{#1}[[{\bf #2}]]}}$
$\newcommand{$\ppxf$}{pPXF}$
$\newcommand{$\oiii$}{\textup{[O $\textsc${iii}]}}$
$\newcommand{$\nii$}{\textup{[N $\textsc${ii}]}}$
$\newcommand{$\sii$}{\textup{[S $\textsc${ii}]}}$
$\newcommand{$\oi$}{\textup{[O $\textsc${i}]}}$
$\newcommand{$\sii$i}{\textup{[S $\textsc${iii}]}}$
$\newcommand{$\oi$i}{\textup{[O $\textsc${ii}]}}$
$\newcommand{$\hii$}{\textup{H} $\textsc${ii}}$
$\newcommand{$\ha$}{\textup{H}$\ensuremath${\alpha}}$
$\newcommand{$\hb$}{\textup{H}$\ensuremath${\beta}}$
$\newcommand{$\re$}{\textup{R}$\ensuremath${_\mathrm{e}}}$
$\newcommand{$\Te$}{\textup{T}$\ensuremath${_\mathrm{e}}}$
$\newcommand{$\SFR$}{$\ensuremath${\Sigma_{\mathrm{SFR}}}}$
$\newcommand{$\aha$}{$\ensuremath${A_{\mathrm{ H \alpha}}}}$</div>



<div id="title">

#  maps

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.08487-b31b1b.svg)](https://arxiv.org/abs/2211.08487)<mark>Appeared on: 2022-11-17</mark> - _accepted in A&A_

</div>
<div id="authors">

Francesco Belfiore, et al. -- incl., <mark>Kathryn Kreckel</mark>, <mark>Lukas Neumann</mark>, <mark>Eva Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** Mapping star-formation rates (SFR) within galaxies is key to unveiling their assembly and evolution. Calibrations exist for computing SFR from a combination of ultraviolet andinfrared bands for galaxies as integrated systems, but their applicability to sub-galactic (kpc) scales remains largely untested. Here we use integral field spectroscopy of 19 nearby ($D <$20 Mpc) galaxies obtained by PHANGS--MUSE to derive accurate Balmer decrements (H$\alpha$/H$\beta$) and attenuation-corrected H$\alpha$maps. We combine this information with mid-infrared maps from\textit{WISE}at 22$\rm \mu m$, and ultraviolet maps from\textit{GALEX}in the far-UV band, to derive SFR surface densities in nearby galaxies on resolved (kpc) scales. Using the$\ha$attenuation-corrected SFR as a reference, we find that hybrid recipes from the literature overestimate the SFR in regions of low SFR surface density, low specific star-formation rate (sSFR), low attenuation and old stellar ages. We attribute these trends to heating of the dust by old stellar populations (`IR cirrus'). We calibrate this effect by proposing functional forms for the coefficients in front of the IR term which depend on band ratios sensitive to the sSFR. Such recipes return SFR estimates in agreement with those in the literature at high sSFR ($\rm log(sSFR/yr^{-1}) > -9.9$). Moreover, they lead to negligible bias and$<$0.16 dex scatter when compared to our reference attenuation-corrected SFR from$\ha$. These calibrations prove reliable as a function of physical scale. In particular, they agree within 10\%with the attenuation corrections computed from the Balmer decrement on 100 pc scales. Despite small quantitative differences, our calibrations are also applicable to integrated galaxy scales probed by the MaNGA survey, albeit with a larger scatter (up to 0.22 dex). Observations with\textit{JWST}open up the possibility to calibrate these relations in nearby galaxies with cloud-scale ($\sim$100 pc) resolution mid-IR imaging.

</div>

<div id="div_fig1">

<img src="tmp_2211.08487/./Fig/Fig5.png" alt="Fig6" width="100%"/>

**Figure 6. -** The hybridisation coefficients of W4 with either FUV ($\rm C^{FUV}_{W4}$, Panels a, b, c) or $\ha$($\rm C^{H\alpha}_{W4}$, Panels d, e) as a function of sSFR (left), $L_{\rm H\alpha}/L_{\rm W1}$(middle) and $L_{\rm FUV}/L_{\rm W1}$(right). These coefficients are calibrated to match the SFR obtained from Balmer-decrement-corrected $\ha$ for individual 15$\arcsec$($\sim$ kpc-scale) regions (grey dots). Red hexagons and shaded areas show the median trends and scatter. Median relations are fitted with a broken power law (equation $\re$f{eq:hybrid_ratios}) and the best-fit model is shown as a dashed blue line. The scatter $\sigma$ of the data with respect to the best-fit is presented in the bottom-right. Values of $C_{\rm W4}^{\rm FUV}$ and $C_{\rm W4}^{\rm H\alpha}$ from the literature are shown as black dashed horizontal lines. Our data for kpc-scale regions agree well with the literature values at high sSFR, but show a systematic deviation for lower sSFR which is captured well by the power-law model. (*fig:fig4*)

</div>
<div id="div_fig2">

<img src="tmp_2211.08487/./Fig/Fig7.png" alt="Fig7" width="100%"/>

**Figure 7. -** The hybridization coefficients of W4 with either FUV (Panels a, b, c) or $\ha$(Panels d, e) as a function of sSFR (left), $L_{\rm H\alpha}/L_{\rm W1}$(middle) and $L_{\rm FUV}/L_{\rm W1}$(right) for the MaNGA sample (using galaxy-integrated fluxes). Best-fit relations obtained for the PHANGS 15$\arcsec$ data (Fig. $\re$f{fig:fig4}) are shown in blue. Blue diamonds show the results obtained considering the integrated fluxes from the 19 PHANGS galaxies. Values of $C_{\rm W4}^{\rm FUV}$ and $C_{\rm W4}^{\rm H\alpha}$ from the literature are shown as black dashed horizontal lines and labelled in the figure (see text for additional detail). The scatter $\sigma$ of the data with respect to the best-fit is presented in the bottom-right. The MaNGA data show similar trends as the resolved regions in PHANGS galaxies, even though in general it displays flatter slopes and does not reach the same values of $C$ at the highest sSFR.  (*fig:fig6*)

</div>
<div id="div_fig3">

<img src="tmp_2211.08487/./Fig/Fig4.png" alt="Fig5" width="100%"/>

**Figure 5. -** Comparison between $E(B-V)$ estimated via the Balmer Decrement and the same quantity calculated via FUV+W4 (left) and $\ha$+W4 (right), colour-coded by sSFR. In the left panel we show as a dashed line the ratio between the differential reddening of gas and stars inferred by \cite{Calzetti2000}, $\rm E(B-V)_{stars} = 0.44  E(B-V)_{gas}$. Solid black lines in both panels represent the one-to-one line. In both panels attenuation is overestimated in regions of low sSFR because of IR cirrus. (*fig:fig3*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\argmax}{\operatornamewithlimits{argmax}}$
$\newcommand{\argmin}{\operatornamewithlimits{argmin}}$
$\newcommand{\lae}{\ensuremath{\mathrm{Ly}\alpha} emitter}$
$\newcommand{\laes}{\ensuremath{\mathrm{Ly}\alpha} emitters}$
$\newcommand{\vmpc}{\ensuremath{\mathrm{cMpc^{3}}}}$
$\newcommand{\invmpc}{\ensuremath{\mathrm{cMpc^{-3}}}}$
$\newcommand{\msun}{\ifmmode M_{\odot} \else M_{\odot}\fi}$
$\newcommand{\msunyr}{\ensuremath{\mathrm{M_{\odot} yr^{-1}}}}$
$\newcommand{\kms}{\ensuremath{\mathrm{km s^{-1}}}}$
$\newcommand{\degree}{\ensuremath{^\circ}}$
$\newcommand{\ergslum}{\ensuremath{\mathrm{erg s^{-1}}}}$
$\newcommand{\ergsluma}[1]{\ensuremath{\mathrm{10^{#1} erg s^{-1}}}}$
$\newcommand{\ergslumb}[2]{\ensuremath{\mathrm{{#1} \times 10^{#2}   erg s^{-1}}}}$
$\newcommand{\ergs}{\ensuremath{\mathrm{erg s^{-1} cm^{-2} \AA^{-1}}}}$
$\newcommand{\ergsa}[1]{\ensuremath{\mathrm{10^{#1} erg s^{-1} cm^{-2} \AA^{-1}}}}$
$\newcommand{\ergsb}[2]{\ensuremath{\mathrm{{#1} \times 10^{#2} erg s^{-1} cm^{-2} \AA^{-1}}}}$
$\newcommand{\ergsline}{\ensuremath{\mathrm{erg s^{-1} cm^{-2}}}}$
$\newcommand{\ergslineb}[2]{\ensuremath{\mathrm{#1 \times 10^{#2} erg s^{-1} cm^{-2}}}}$
$\newcommand{\ergslinea}[1]{\ensuremath{\mathrm{10^{#1} erg s^{-1} cm^{-2}}}}$
$\newcommand{\erglinesurf}[2]{\ensuremath{\mathrm{#1 \times 10^{#2} erg s^{-1} cm^{-2} arcsec^{-2}}}}$
$\newcommand{\erglsurf}[1]{\ensuremath{\mathrm{10^{#1} erg s^{-1} cm^{-2} arcsec^{-2}}}}$
$\newcommand{\ergsurfb}{\ensuremath{\mathrm{erg s^{-1} cm^{-2} arcsec^{-2}}}}$
$\newcommand{\ergsurf}{\ensuremath{\mathrm{erg s^{-1} cm^{-2} \AA^{-1} arcsec^{-2}}}}$
$\newcommand{\lya}{Ly\textalpha}$
$\newcommand{\ha}{H\textalpha}$
$\newcommand{\hb}{H\textbeta}$
$\newcommand{\hg}{H\textgamma}$
$\newcommand{\hd}{H\textdelta}$
$\newcommand{\oii}{[O {\sc ii}]}$
$\newcommand{\oiid}{[O {\sc ii}]\textlambda\textlambda3726,3729}$
$\newcommand{\oiiid}{[O {\sc iii}]\textlambda4959,5007}$
$\newcommand{\oiiia}{[O {\sc iii}]\textlambda4959}$
$\newcommand{\oiiib}{[O {\sc iii}]\textlambda5007}$
$\newcommand{\ciii}{C {\sc iii}]}$
$\newcommand{\ciiid}{C {\sc iii}]\textlambda\textlambda1907,1909}$
$\newcommand{\civ}{C {\sc iv}}$
$\newcommand{\civd}{C {\sc iv} \textlambda\textlambda1548,1550}$
$\newcommand{\mgii}{Mg {\sc ii}}$
$\newcommand{\mgiid}{Mg {\sc ii} \textlambda\textlambda2797,2803}$
$\newcommand{\heii}{He {\sc ii}}$
$\newcommand{\mosaic}{\textsf{MOSAIC}}$
$\newcommand{\udft}{\textsf{UDF-10}}$
$\newcommand{\mxdf}{\textsf{MXDF}}$
$\newcommand{\drt}{\textsf{DR2}}$
$\newcommand{\dro}{\textsf{DR1}}$
$\newcommand{\origin}{\textsf{ORIGIN}}$
$\newcommand{\odhin}{\textsf{ODHIN}}$
$\newcommand{\nbext}{\textsf{NBEXT}}$
$\newcommand{\pymarz}{\textsf{pyMarZ}}$
$\newcommand{\mpdaf}{\textsf{MPDAF}}$
$\newcommand{\pfit}{\textsf{pyPlatefit}}$
$\newcommand{\sex}{\textsf{SExtractor}}$
$\newcommand{\se}{\textsf{SourceInspector}}$
$\newcommand{\psfrec}{\textsf{muse-psfr}}$
$\newcommand{\prospector}{\textsf{Prospector}}$
$\newcommand{\magphys}{\textsf{Magphys}}$
$\newcommand{\amused}{\textsf{AMUSED}}$
$\newcommand{\hubble}{\textsf{\textit{Hubble}}}$
$\newcommand{\hudf}{\textsf{HUDF}}$
$\newcommand{\uvudf}{\textsf{UVUDF}}$
$\newcommand{\tdhst}{\textsf{3D-HST}}$
$\newcommand{\candels}{\textsf{CANDELS v2}}$
$\newcommand{\astrod}{\textsf{ASTRODEEP}}$
$\newcommand{\ALG@name}{Algorithm}$
$\newcommand{\b0}{\boldsymbol{0}}$
$\newcommand{\ba}{\boldsymbol{a}}$
$\newcommand{\bb}{\boldsymbol{b}}$
$\newcommand{\bd}{\boldsymbol{d}}$
$\newcommand{\bg}{\boldsymbol{g}}$
$\newcommand{\bs}{\boldsymbol{s}}$
$\newcommand{\bt}{\boldsymbol{t}}$
$\newcommand{\bu}{\boldsymbol{u}}$
$\newcommand{\bv}{\boldsymbol{v}}$
$\newcommand{\bx}{\boldsymbol{x}}$
$\newcommand{\by}{\boldsymbol{y}}$
$\newcommand{\bz}{\boldsymbol{z}}$
$\newcommand{\bA}{\boldsymbol{A}}$
$\newcommand{\bB}{\boldsymbol{B}}$
$\newcommand{\bC}{\boldsymbol{C}}$
$\newcommand{\bD}{\boldsymbol{D}}$
$\newcommand{\bE}{\boldsymbol{E}}$
$\newcommand{\bF}{\boldsymbol{F}}$
$\newcommand{\bG}{\boldsymbol{G}}$
$\newcommand{\bH}{\boldsymbol{H}}$
$\newcommand{\bI}{\boldsymbol{I}}$
$\newcommand{\bL}{\boldsymbol{L}}$
$\newcommand{\bM}{\boldsymbol{M}}$
$\newcommand{\bP}{\boldsymbol{P}}$
$\newcommand{\bQ}{\boldsymbol{Q}}$
$\newcommand{\bR}{\boldsymbol{R}}$
$\newcommand{\bS}{\boldsymbol{S}}$
$\newcommand{\bT}{\boldsymbol{T}}$
$\newcommand{\bU}{\boldsymbol{U}}$
$\newcommand{\wbU}{\widetilde{\boldsymbol{U}}}$
$\newcommand{\bV}{\boldsymbol{V}}$
$\newcommand{\bX}{\boldsymbol{X}}$
$\newcommand{\bY}{\boldsymbol{Y}}$
$\newcommand{\bZ}{\boldsymbol{Z}}$
$\newcommand{\beps}{\boldsymbol{\epsilon}}$
$\newcommand{\bSigma}{\boldsymbol{\Sigma}}$
$\newcommand{\bbeta}{\boldsymbol{\beta}}$
$\newcommand{\br}{\boldsymbol{r}}$
$\newcommand{\bw}{\boldsymbol{w}}$
$\newcommand{\bW}{\boldsymbol{W}}$
$\newcommand{\Fm}{F^{-\textrm{min}}}$
$\newcommand{\FM}{F^{\textrm{max}}}$
$\newcommand{\}{mean}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\argmax$}{\operatornamewithlimits{argmax}}$
$\newcommand{$\argmin$}{\operatornamewithlimits{argmin}}$
$\newcommand{$\lae$}{$\ensuremath${\mathrm{Ly}\alpha} emitter}$
$\newcommand{$\lae$s}{$\ensuremath${\mathrm{Ly}\alpha} emitters}$
$\newcommand{$\vmpc$}{$\ensuremath${\mathrm{cMpc^{3}}}}$
$\newcommand{$\invmpc$}{$\ensuremath${\mathrm{cMpc^{-3}}}}$
$\newcommand{$\msun$}{\ifmmode M_{\odot} \else M_{\odot}\fi}$
$\newcommand{$\msun$yr}{$\ensuremath${\mathrm{M_{\odot} yr^{-1}}}}$
$\newcommand{$\kms$}{$\ensuremath${\mathrm{km s^{-1}}}}$
$\newcommand{$\degree$}{$\ensuremath${^\circ}}$
$\newcommand{$\ergslum$}{$\ensuremath${\mathrm{erg s^{-1}}}}$
$\newcommand{$\ergslum$a}[1]{$\ensuremath${\mathrm{10^{#1} erg s^{-1}}}}$
$\newcommand{$\ergslum$b}[2]{$\ensuremath${\mathrm{{#1} \times 10^{#2}   erg s^{-1}}}}$
$\newcommand{$\ergs$}{$\ensuremath${\mathrm{erg s^{-1} cm^{-2} \AA^{-1}}}}$
$\newcommand{$\ergs$a}[1]{$\ensuremath${\mathrm{10^{#1} erg s^{-1} cm^{-2} \AA^{-1}}}}$
$\newcommand{$\ergs$b}[2]{$\ensuremath${\mathrm{{#1} \times 10^{#2} erg s^{-1} cm^{-2} \AA^{-1}}}}$
$\newcommand{$\ergs$line}{$\ensuremath${\mathrm{erg s^{-1} cm^{-2}}}}$
$\newcommand{$\ergs$lineb}[2]{$\ensuremath${\mathrm{#1 \times 10^{#2} erg s^{-1} cm^{-2}}}}$
$\newcommand{$\ergs$linea}[1]{$\ensuremath${\mathrm{10^{#1} erg s^{-1} cm^{-2}}}}$
$\newcommand{$\erglinesurf$}[2]{$\ensuremath${\mathrm{#1 \times 10^{#2} erg s^{-1} cm^{-2} arcsec^{-2}}}}$
$\newcommand{$\erglsurf$}[1]{$\ensuremath${\mathrm{10^{#1} erg s^{-1} cm^{-2} arcsec^{-2}}}}$
$\newcommand{$\ergs$urfb}{$\ensuremath${\mathrm{erg s^{-1} cm^{-2} arcsec^{-2}}}}$
$\newcommand{$\ergs$urf}{$\ensuremath${\mathrm{erg s^{-1} cm^{-2} \AA^{-1} arcsec^{-2}}}}$
$\newcommand{$\lya$}{Ly\textalpha}$
$\newcommand{$\ha$}{H\textalpha}$
$\newcommand{$\hb$}{H\textbeta}$
$\newcommand{$\hg$}{H\textgamma}$
$\newcommand{$\hd$}{H\textdelta}$
$\newcommand{$\oii$}{[O {\sc ii}]}$
$\newcommand{$\oii$d}{[O {\sc ii}]\textlambda\textlambda3726,3729}$
$\newcommand{$\oii$id}{[O {\sc iii}]\textlambda4959,5007}$
$\newcommand{$\oii$ia}{[O {\sc iii}]\textlambda4959}$
$\newcommand{$\oii$ib}{[O {\sc iii}]\textlambda5007}$
$\newcommand{$\ciii$}{C {\sc iii}]}$
$\newcommand{$\ciii$d}{C {\sc iii}]\textlambda\textlambda1907,1909}$
$\newcommand{$\civ$}{C {\sc iv}}$
$\newcommand{$\civ$d}{C {\sc iv} \textlambda\textlambda1548,1550}$
$\newcommand{$\mgii$}{Mg {\sc ii}}$
$\newcommand{$\mgii$d}{Mg {\sc ii} \textlambda\textlambda2797,2803}$
$\newcommand{$\heii$}{He {\sc ii}}$
$\newcommand{$\mosaic$}{\textsf{MOSAIC}}$
$\newcommand{$\udft$}{\textsf{UDF-10}}$
$\newcommand{$\mxdf$}{\textsf{MXDF}}$
$\newcommand{$\drt$}{\textsf{DR2}}$
$\newcommand{$\dro$}{\textsf{DR1}}$
$\newcommand{$\origin$}{\textsf{ORIGIN}}$
$\newcommand{$\odhin$}{\textsf{ODHIN}}$
$\newcommand{$\nbext$}{\textsf{NBEXT}}$
$\newcommand{$\pymarz$}{\textsf{pyMarZ}}$
$\newcommand{$\mpdaf$}{\textsf{MPDAF}}$
$\newcommand{$\pfit$}{\textsf{pyPlatefit}}$
$\newcommand{$\sex$}{\textsf{SExtractor}}$
$\newcommand{$\se$}{\textsf{SourceInspector}}$
$\newcommand{$\psfrec$}{\textsf{muse-psfr}}$
$\newcommand{$\prospector$}{\textsf{Prospector}}$
$\newcommand{$\magphys$}{\textsf{Magphys}}$
$\newcommand{$\amused$}{\textsf{AMUSED}}$
$\newcommand{$\hubble$}{\textsf{\textit{Hubble}}}$
$\newcommand{$\hudf$}{\textsf{HUDF}}$
$\newcommand{$\uvudf$}{\textsf{UVUDF}}$
$\newcommand{$\tdhst$}{\textsf{3D-HST}}$
$\newcommand{$\candels$}{\textsf{CANDELS v2}}$
$\newcommand{$\astrod$}{\textsf{ASTRODEEP}}$
$\newcommand{$\ALG$@name}{Algorithm}$
$\newcommand{$\b$0}{$\b$oldsymbol{0}}$
$\newcommand{$\b$a}{$\b$oldsymbol{a}}$
$\newcommand{$\b$b}{$\b$oldsymbol{b}}$
$\newcommand{$\b$d}{$\b$oldsymbol{d}}$
$\newcommand{$\b$g}{$\b$oldsymbol{g}}$
$\newcommand{$\b$s}{$\b$oldsymbol{s}}$
$\newcommand{$\b$t}{$\b$oldsymbol{t}}$
$\newcommand{$\b$u}{$\b$oldsymbol{u}}$
$\newcommand{$\b$v}{$\b$oldsymbol{v}}$
$\newcommand{$\b$x}{$\b$oldsymbol{x}}$
$\newcommand{$\b$y}{$\b$oldsymbol{y}}$
$\newcommand{$\b$z}{$\b$oldsymbol{z}}$
$\newcommand{$\b$A}{$\b$oldsymbol{A}}$
$\newcommand{$\b$B}{$\b$oldsymbol{B}}$
$\newcommand{$\b$C}{$\b$oldsymbol{C}}$
$\newcommand{$\b$D}{$\b$oldsymbol{D}}$
$\newcommand{$\b$E}{$\b$oldsymbol{E}}$
$\newcommand{$\b$F}{$\b$oldsymbol{F}}$
$\newcommand{$\b$G}{$\b$oldsymbol{G}}$
$\newcommand{$\b$H}{$\b$oldsymbol{H}}$
$\newcommand{$\b$I}{$\b$oldsymbol{I}}$
$\newcommand{$\b$L}{$\b$oldsymbol{L}}$
$\newcommand{$\b$M}{$\b$oldsymbol{M}}$
$\newcommand{$\b$P}{$\b$oldsymbol{P}}$
$\newcommand{$\b$Q}{$\b$oldsymbol{Q}}$
$\newcommand{$\b$R}{$\b$oldsymbol{R}}$
$\newcommand{$\b$S}{$\b$oldsymbol{S}}$
$\newcommand{$\b$T}{$\b$oldsymbol{T}}$
$\newcommand{$\b$U}{$\b$oldsymbol{U}}$
$\newcommand{$\wbU$}{\widetilde{$\b$oldsymbol{U}}}$
$\newcommand{$\b$V}{$\b$oldsymbol{V}}$
$\newcommand{$\b$X}{$\b$oldsymbol{X}}$
$\newcommand{$\b$Y}{$\b$oldsymbol{Y}}$
$\newcommand{$\b$Z}{$\b$oldsymbol{Z}}$
$\newcommand{$\b$eps}{$\b$oldsymbol{\epsilon}}$
$\newcommand{$\b$Sigma}{$\b$oldsymbol{\Sigma}}$
$\newcommand{$\b$beta}{$\b$oldsymbol{$\b$eta}}$
$\newcommand{$\b$r}{$\b$oldsymbol{r}}$
$\newcommand{$\b$w}{$\b$oldsymbol{w}}$
$\newcommand{$\b$W}{$\b$oldsymbol{W}}$
$\newcommand{$\Fm$}{F^{-\textrm{min}}}$
$\newcommand{$\FM$}{F^{\textrm{max}}}$
$\newcommand{\}{mean}$</div>



<div id="title">

# Based on observations made with ESO telescopes at the La Silla Paranal Observatory under the programs 094.A-0289(B), 095.A-0010(A), 096.A-0045(A), 096.A-0045(B) and 1101.A-0127

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.08493-b31b1b.svg)](https://arxiv.org/abs/2211.08493)<mark>Appeared on: 2022-11-17</mark> - _46 pages, 48 figures_

</div>
<div id="authors">

Roland Bacon, et al. -- incl., <mark><mark>Leindert Boogaard</mark></mark>, <mark><mark>Yucheng Guo</mark></mark>, <mark><mark>Christian Herenz</mark></mark>

</div>
<div id="abstract">

**Abstract:** We present the second data release of the MUSE\textit{Hubble}Ultra-Deep Field surveys, which includes the deepest spectroscopic survey ever performed. The MUSE data, with their 3D content, amazing depth, wide spectral range, and excellent spatial and medium spectral resolution, are rich in information. Their location in the\textit{Hubble}ultra-deep field area, which benefits from an exquisite collection of ancillary panchromatic information, is a major asset. This update of the first release incorporates a new 141-hour adaptive-optics-assisted MUSE eXtremely Deep Field ($\mxdf$; 1 arcmin diameter field of view) in addition to the reprocessed 10-hour mosaic ($\rm 3 \times 3   arcmin^2$) and the single 31-hour deep field ($\rm 1 \times 1   arcmin^2$). All three data sets were processed and analyzed homogeneously using advanced data reduction and analysis methods. The$3\sigma$point-source flux limit of an unresolved emission line reaches$\rm 3.1 \times 10^{-19}$and$\rm 6.3 \times 10^{-20}   $\ergs$line$at 10- and 141-hour depths, respectively.We have securely identified and measured the redshift of 2221 sources, an increase of 41\%compared to the first release. With the exception of eight stars, the collected sample consists of 25 nearby galaxies ($z < 0.25$), 677$\oii$emitters ($z=0.25-1.5$), 201 galaxies in the MUSE  redshift desert range ($z=1.5-2.8$), and 1308$\lae$s($z=2.8-6.7$).This represents an order of magnitude more redshifts than the collection of all spectroscopic redshifts obtained before MUSE in the\textit{Hubble}ultra-deep field area (i.e., 2221 versus 292). At high redshift ($z > 3$), the difference is even more striking, with a factor of 65 increase (1308 versus 20).We compared the measured redshifts against three published photometric redshift catalogs and find the photo-z accuracy  to be lower than the constraints provided by photo-z fitting codes.Eighty percent of the galaxies in our final catalog have an HST counterpart. These galaxies are on average faint, with a median AB F775W magnitude of 25.7 and 28.7 for the$\oii$and$\lya$emitters, respectively. Fits of their spectral energy distribution show that these galaxies tend to be low-mass star-forming galaxies, with a median stellar mass of$\rm 6.2 \times 10^8   M_\odot$and a median star-formation rate of$\rm 0.4   M_\odot yr^{-1}$.We measured the completeness of our catalog with respect to HST and found that, in the deepest 141-hour  area, 50\%completeness is achieved for an AB magnitude of 27.6 and 28.7 (F775W) at$z=0.8-1.6$and$z=3.2-4.5$, respectively.Twenty percent of our catalog, or 424 galaxies, have no HST counterpart. The vast majority of these new sources are high equivalent-width$z>2.8$\laesthat are detected by MUSE thanks to their bright and asymmetric broad$\lya$line.We release advanced data products, specific software, and a web interface to select and download data sets.

</div>

<div id="div_fig1">

<img src="tmp_2211.08493/paper_figures/src7636-6294.png" alt="Fig15" width="100%"/>

**Figure 15. -** Example of $\odhin$ de-blending of two close sources MID-7636 (RID-25333), a $z=0.5$ star forming galaxy, and MID-6294 (RID-22260), a $z=5.5$\lae. The separation of the sources is 0.4 arcsec. The top row of panels shows, from left to right: the HST F775W, MUSE white-light, [OIII]5007\AA and $\lya$ narrow band images ($\rm 5 \times 5   arcsec^2$ size). The central and bottom panels display the spectra derived from $\odhin$(in blue) and $\origin$ extraction (in orange) for each source. Fluxes and observed wavelengths are expressed in units of $\ergs$a{-20} and \AA, respectively. (*fig:deblend_example*)

</div>
<div id="div_fig2">

<img src="tmp_2211.08493/paper_figures/skyspec.png" alt="Fig1" width="100%"/>

**Figure 1. -** MUSE spectral range. The typical sky spectrum (shown in black) is observed in one 25 min observing block. The blue shaded wavelength region shows the location of the sodium notch filter used in MUSE GLAO mode. The response curves of the HST ACS filters F606W, F775W, and F850LP are also indicated.
 (*fig:specrange*)

</div>
<div id="div_fig3">

<img src="tmp_2211.08493/paper_figures/process.png" alt="Fig10" width="100%"/>

**Figure 10. -** Schematic of the processes involved in the data release production.
 (*fig:process*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

93  publications in the last 7 days.
	 _build/html/2211.08493.md
	 _build/html/2211.08487.md
	 _build/html/2211.08294.md
	 _build/html/2211.07667.md
	 _build/html/2211.07659.md
	 _build/html/2211.06454.md
	 _build/html/2211.06205.md
	 _build/html/2211.06194.md
	 _build/html/2211.05022.md
	 _build/html/2211.04048.md
	 _build/html/2211.03641.md
	 _build/html/2211.02502.md
	 _build/html/2211.01474.md
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/html/2211.00010.md
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers